## Inquery_type만 전처리 했을때 Baseline과 비교

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

/tmp/ipykernel_211493/3824362204.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [7]:
data_path = "/home/workspace/LGamiers/"
df_train = pd.read_csv(data_path + "train.csv") # 학습용 데이터
df_test = pd.read_csv(data_path + "submission.csv")

In [8]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)
    # unique value 찾고 아스키코드 기준으로 sort
    # my_dict을 통해 unique value에 대응하는 label 생성 
    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    # my_dict의 index와 매치되는 series값을 my_dict의 value로 변환한다.
    series = series.map(my_dict)

    return series

In [9]:
def inquiry_type_preprosessing(df):
    # inquiry_type feature 전처리 
    df['inquiry_type'] = df['inquiry_type'].str.lower()
    # 특수문자를 대체 문자열 지정
    replacement = {'/': ' ', '-':' ', '_':' '}
    # replace() 함수를 사용하여 특수 문자 대체
    df['inquiry_type'].replace(replacement, regex=True, inplace=True)
    # value count의 값이 1개인 feature name extraction (해당 방식 토의) 
    inquiry_series = df['inquiry_type'].value_counts()
    replace_feature = []
    for idx, feature in enumerate(inquiry_series.index):
        if inquiry_series[idx] == 1:
            replace_feature.append(feature)
    # 총 39개의 데이터 Concat
    df['inquiry_type'] = df['inquiry_type'].apply(lambda x: 'aimers_0203' if x in replace_feature else x)
    return df

In [10]:
# 레이블 인코딩할 칼럼들
label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

df_all = pd.concat([df_train[label_columns], df_test[label_columns]])
df_all = inquiry_type_preprosessing(df_all)
for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

/tmp/ipykernel_211493/3540704721.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['inquiry_type'].replace(replacement, regex=True, inplace=True)
/tmp/ipykernel_211493/3540704721.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if inquiry_series[idx] == 1:


In [11]:
# 위에서 전처리할때는 한번에 concat해서 진행하고 분리한다. 
for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]

In [12]:
x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,)

In [13]:
model = DecisionTreeClassifier()
# 결측치를 0으로 처리
model.fit(x_train.fillna(0), y_train)

DecisionTreeClassifier()

In [14]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [15]:
pred = model.predict(x_val.fillna(0))
get_clf_eval(y_val, pred)

오차행렬:
 [[  742   205]
 [  250 10663]]

정확도: 0.9616
정밀도: 0.7480
재현율: 0.7835
F1: 0.7653


In [ ]:
# Baseline 
# 오차행렬:
#  [[  734   213]
#  [  248 10665]]

# 정확도: 0.9611
# 정밀도: 0.7475
# 재현율: 0.7751
# F1: 0.7610


# 카테고리를 조금 더 줄인다면 가능성이 보임